In [1]:
# Importer les bibliotheques a utiliser
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.integrate import trapz
import os

In [42]:
path = ("/Users/sanaailla/Desktop/Intern/Data/mars/")
patients = pd.read_csv(path + "dir_list.csv",header=None)
## Les types des events qui existent
ac,ao,hc,ho = 0,1,2,3
patients

,0
0,PA1011
1,PA1089
2,PA1123
3,PA1135
4,PA1141
5,PA1150
6,PA1154
7,PA1171
8,PA1178
9,PA1185


In [34]:
def sat_annot_desats(patient_name, path):
    saturations_path = os.path.join(path, patient_name, "saturations.csv")
    annotations_path = os.path.join(path, patient_name, "Annotations.csv")
    desaturation_path = os.path.join(path, patient_name, "Desaturation_3.csv")

    if  not os.path.isfile(desaturation_path) or not os.path.isfile(saturations_path) or not os.path.isfile(annotations_path):
        print(patient_name)
        return None,None,None
    else :
        saturations = pd.read_csv(saturations_path, index_col=0)
        annotations = pd.read_csv(annotations_path,index_col= 0)
        events =  ["Apnée Centrale","hypopnée Centrale","Apnée Obstructive","Hypopnée"]
        annotations = annotations[annotations["Evènement"].isin(events)][["Heure","Evènement","Durée"]]
        annotations["Heure"] = annotations["Heure"].astype(int)
        annotations["Durée"] = annotations["Durée"].astype(int)
        desaturations = pd.read_csv(desaturation_path,index_col=0)
        # saturations["Heure"] = saturations["Heure"] .astype("int")
        # saturations = saturations.groupby("Heure").mean().reset_index()
        saturations = edf.filtrer_signalSat(saturations)
    return saturations, annotations,desaturations

In [ ]:
# Baseline
def baseline(sig):
    # if (method == "mean"):
    #     m = np.mean(sig)
    # else :
    #     m = np.quantile(sig[:-1][(np.abs(np.diff(sig)) == 0)],0.8)
    m = sig[0]
    return m

In [ ]:
def annotate_sig(saturations, annotations):
    saturations["Évenement"] = np.NAN
    saturations["Durée"] = np.NAN
    for heure, duree, evenement in zip(annotations.Heure, annotations.Durée, annotations.Evènement):
        ix = saturations[saturations["Heure"] == heure].index.values
        if len(ix)!=0:
            i = ix[0]
            saturations.loc[i:i + duree - 1, "Évenement"] = evenement
            saturations.loc[i:i + duree - 1, "Durée"] = duree
    return saturations

In [ ]:
def desaturations_lab(desaturations, annotations, seuil = float("inf")) :
    debut_desat = desaturations["HEURE_debut"].astype(int) + 1
    debut_label = annotations.Heure
    events = annotations.Evènement
    duree_label = annotations.Durée
    labels = []
    closest_times = []
    differences = []
    for i in range(len(desaturations)):
        closest_label = np.NaN
        closest_time = np.NaN
        diff = np.NaN
        for label_time,event,duree in zip(debut_label,events,duree_label):
            if label_time < debut_desat[i] and (debut_desat[i] - label_time) < seuil + duree  and label_time not in closest_times:
                closest_label = event
                closest_time = label_time
                diff =  debut_desat[i] - label_time
        differences.append(diff)
        labels.append(closest_label)
        closest_times.append(closest_time)
    desaturations["events"] = labels
    desaturations["difference"] = differences
    desaturations["closest time"] = closest_times
    desaturations = desaturations[~desaturations["events"].isna()].reset_index(drop=True)
    desaturations["events"] =  desaturations["events"].replace({"hypopnée Centrale": hc,"Hypopnée": ho,"Apnée Obstructive": ao,"Apnée Centrale": ac})
    return desaturations

In [16]:
def compute_area(baseline, desaturations):
    debut_events = desaturations.HEURE_debut.astype(int)
    duree_events = desaturations.DUREE_event.astype(int)
    for ix in range(len(desaturations)):
        if (len(saturations[saturations.Heure == debut_events[ix]])!=0) :
            ix_debut_desat = saturations[saturations.Heure == debut_events[ix]].index.values[0]
            desaturation = saturations["SAT"].iloc[int(ix_debut_desat):int(ix_debut_desat) + duree_events[ix]]
            x = desaturation.index.values
            y = desaturation.values
            # plt.plot(y)
            # plt.axhline(baseline,c = "red")
            # plt.show()
            reference_line = [baseline for i in range(len(desaturation))]
            desaturations.loc[ix, "area_up"] = trapz(np.maximum(y, reference_line) - reference_line, x)
            desaturations.loc[ix, "area_down"] = trapz(np.minimum(y, reference_line) - reference_line, x)
        else :
            continue
    return desaturations.dropna(subset = ["area_up","area_down"])

In [28]:
data = pd.DataFrame()
# for patient in patients[0]:
patient = "PA1203"
saturations, annotations, desaturations = sat_annot_desats(patient,path)
# if (saturations is None):
#     continue
# else :
baselinee = baseline(saturations.SAT)
saturations =  annotate_sig(saturations,annotations)
# On teste par exemple un threshold d'une minute et 30 secondes
seuil = 90
desaturations = desaturations_lab(desaturations,annotations,seuil)
desaturations["area_up"] = np.NAN
desaturations["area_down"] = np.NAN
desaturations["patient"] = patient
desaturations = compute_area(baselinee,desaturations)
data = pd.concat([data,desaturations], axis = 0)

In [35]:
patient = "PA1203"
saturations, annotations, desaturations = sat_annot_desats(patient,path)
saturations

,SAT
0,93.91033
1,94.49856
2,94.58840
3,95.40870
4,94.49856
...,...
25772,95.49854
25773,95.49854
25774,94.78371
25775,97.35789


In [19]:
X = data[["MAX_desat","MIN_desat","MAX_resat","DUREE_event","PENTE_desat","PENTE_resat","area_down","area_up","DELTA_desat","DELTA_resat"]] #all
#X.loc[:,"diff_pente"] = abs(X.loc[:,"PENTE_desat"] - X.loc[:,"PENTE_resat"])
#X = data[["MIN_desat","DUREE_event","PENTE_desat","PENTE_resat","area_down","DELTA_desat","DELTA_resat",]]
Y = data["events"]

In [ ]:
import seaborn as sns
data_all = pd.concat([X,Y], axis = 1)
# splot = sns.pairplot(data_all,hue = "events")

In [40]:
Xa,Ya = X[Y.isin([ac,ao])],Y[Y.isin([ac,ao])]
Xa

,MAX_desat,MIN_desat,MAX_resat,DUREE_event,PENTE_desat,PENTE_resat,area_down,area_up,DELTA_desat,DELTA_resat
1,97.498512,94.498558,97.498512,58.984375,-0.128600,0.084135,-66.137295,4.723435,3.0,2.999954
5,96.498528,92.498589,98.498497,68.671875,-0.249878,0.113928,-207.376123,11.108419,4.0,5.999908
12,95.498543,92.498589,97.498512,76.000000,-0.391437,0.073167,-184.161053,2.433285,3.0,4.999924
36,96.498528,93.498573,96.498528,54.328125,-0.195719,0.076922,-100.472376,0.000000,3.0,2.999954
39,95.498543,92.498589,96.498528,66.000000,-0.130302,0.093073,-173.586765,0.000000,3.0,3.999939
...,...,...,...,...,...,...,...,...,...,...
125,97.498512,94.498558,98.498497,11.000000,-384.000000,0.363889,0.000000,35.274423,3.0,3.999939
126,96.498528,93.498573,99.498482,20.687500,-0.320802,0.529282,0.000000,58.531091,3.0,5.999908
136,99.498482,89.498634,98.498497,44.671875,-0.275328,1.077626,-20.888451,114.890203,10.0,8.999863
154,98.498497,94.498558,97.498512,50.015625,-0.100768,0.290684,-2.235262,185.526757,4.0,2.999954


In [ ]:
## Affichage
#new_desat = features_desat_new[1]
#annotations = annotations_f[1]
#desat = []
#num_plots = len(new_desat)
#num_columns = 3
#num_rows = (num_plots + num_columns - 1) // num_columns
#fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5*num_rows))
#for ix in range(num_plots):
    #ix_debut_lab = int(annotations[annotations["Heure"] == new_desat["temps proche"].iloc[ix]].index.values.min())
    #ix_fin_lab = ix_debut_lab + int(new_desat["duree lab"].iloc[ix])
    #ix_debut_desat = ix_debut_lab + new_desat["difference"].iloc[ix]
    #fin_desat = ix_debut_desat + int(new_desat["DUREE_event"].iloc[ix])
    #ix_debut_diff = int(ix_fin_lab)
    #ix_fin_diff = ix_debut_diff + int(ix_debut_desat - ix_fin_lab)
    #row = ix // num_columns
    #col = ix % num_columns
    #ax = axes[row, col]
    #desat.append(annotations["SAT"].iloc[int(ix_debut_desat):int(fin_desat)+ 1])
    #ax.plot(annotations["SAT"].iloc[ix_debut_lab:ix_fin_lab + 1])
    #ax.plot(annotations["SAT"].iloc[ix_debut_diff:ix_fin_diff + 1], c="green")
    #ax.plot(desat[ix], c="red")
    #ax.axhline(annotations["SAT"].iloc[int(ix_debut_desat)], color="orange", linestyle='dashed')
    #ax.axvline(annotations.iloc[int(ix_debut_desat)].name, color="cyan", linestyle='dashed')
    #ax.axvline(annotations.iloc[int(fin_desat)].name, color="cyan", linestyle='dashed')
    #ax.axhline(min(desat[ix]), color="black", linestyle='dashed')
    #ax.axhline(max(desat[ix]), color="black", linestyle='dashed')
    #type_des = new_desat["labels"].iloc[ix]
    #ax.set_title(f"Desaturation {ix+1} Type :{int(type_des)}")
#plt.tight_layout()
#plt.show()

In [ ]:
data_ap = pd.concat([Xa,Ya], axis = 1)
# splot = sns.pairplot(data_ap,hue = "events")

In [ ]:
for feature in Xa.columns :
    sns.catplot(x = "events", y = feature , data = data_all, kind = "box")

In [ ]:
# Calcul des stats descr pour chaque type d apnee
Xa[Ya == ac].describe()

In [ ]:
Xa[Ya == ao].describe()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xa_scaled = scaler.fit_transform(Xa)
X_train, X_test, Y_train, Y_test = train_test_split(Xa_scaled, Ya, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

In [ ]:
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
train_accuracy = accuracy_score(Y_train, train_predictions)
test_accuracy = accuracy_score(Y_test, test_predictions)
print(f"Training set accuracy: {train_accuracy}")
print(f"Test set accuracy: {test_accuracy}")
train_accuracy_class1 = accuracy_score(Y_train[Y_train == 1], train_predictions[Y_train == 1])
test_accuracy_class1 = accuracy_score(Y_test[Y_test == 1], test_predictions[Y_test == 1])
print(f"Training set accuracy for class 1: {train_accuracy_class1}")
print(f"Test set accuracy for class 1: {test_accuracy_class1}")

In [ ]:
print(Xa.columns)
model.coef_

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
scaler = StandardScaler(with_mean = False) ## to center add with_mean = True
scaled_data = scaler.fit_transform(Xa.loc[:,Xa.columns != "area_up"])
pca = PCA(n_components=2)
transformed_data = pca.fit_transform(scaled_data)
plt.figure(figsize=(10, 5))
plt.scatter(transformed_data[:, 0], transformed_data[:, 1],c = Ya)
plt.show()

In [ ]:
num_clusters = 2
kmeans = KMeans(num_clusters, init="k-means++",n_init=100)
kmeans.fit(Xa.loc[:,Xa.columns != "area_up"])
cluster_labels = kmeans.labels_
plt.figure(figsize=(10, 5))
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], c=cluster_labels)
plt.title('K-means with 2 clusters')
plt.show()


In [ ]:
import sklearn.mixture as skm
mod = skm.GaussianMixture(n_components= 2).fit(Xa.loc[:,Xa.columns != "area_up"])
lab = mod.predict(Xa.loc[:,Xa.columns != "area_up"])
plt.figure(figsize=(10, 5))
plt.scatter(transformed_data[:, 0], transformed_data[:, 1],c = lab*-1)
plt.show()